In [144]:
import time
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests

import pandas as pd

In [145]:
def scrape_table(url, id):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome('C:\Program Files\ChromeDriver\chromedriver.exe',options=options)
    driver.get(url)
    time.sleep(2)
    page = driver.page_source
    soup = bs4.BeautifulSoup(page, 'html.parser')
    driver.quit()

    table = soup.findAll("div", {"id": id, 'class':'table_container'})[0].find('tbody')
    rows = table.findAll('tr')
    head = soup.findAll("div", {"id": id, 'class':'table_container'})[0].find('thead').find('tr')

    dict = {'Year':[]}
    col_headers = []

    for entry in head:
        if len(entry.get_text().strip())>0 and entry.get_text().strip()!='Rk':
            header = entry.get_text()
            if header in col_headers:
                header+='.1'
            dict[header]=[]
            col_headers.append(header)

    for row in rows:
        col = row.findAll('td')
        if len(col)>0:
            dict['Year'].append('2021')
            for i in range(0,len(col_headers)):
                dict[col_headers[i]].append(col[i].get_text())
    
    return pd.DataFrame.from_dict(dict)

In [146]:
passing = scrape_table('https://www.pro-football-reference.com/years/2021/', 'div_passing')
rushing = scrape_table('https://www.pro-football-reference.com/years/2021/', 'div_rushing')

X:\Conda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """


In [147]:
passing.head()

,Year,Tm,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,...,Y/G,Rate,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,EXP
0,2021,Tampa Bay Buccaneers,11,316,466,67.8,3365,30,6.4,9,...,305.9,103.0,15,105,3.1,7.0,7.4,2,3,124.29
1,2021,Las Vegas Raiders,11,278,413,67.3,3262,17,4.1,9,...,296.5,97.3,25,152,5.7,7.4,7.3,1,3,97.31
2,2021,Dallas Cowboys,11,291,422,69.0,3216,24,5.7,8,...,292.4,103.5,20,119,4.5,7.3,7.5,2,3,113.47
3,2021,Kansas City Chiefs,11,306,466,65.7,3195,25,5.4,11,...,290.5,94.3,20,103,4.1,6.6,6.6,2,2,108.39
4,2021,Los Angeles Rams,11,268,406,66.0,3178,27,6.7,10,...,288.9,103.1,17,145,4.0,7.5,7.7,2,2,124.22


In [148]:
rushing.head()

,Year,Tm,G,Att,Yds,TD,Lng,Y/A,Y/G,Fmb,EXP
0,2021,Philadelphia Eagles,12,371,1895,18,31,5.1,157.9,15,49.04
1,2021,Cleveland Browns,12,346,1765,17,70,5.1,147.1,16,39.57
2,2021,Indianapolis Colts,12,335,1734,17,83,5.2,144.5,16,44.35
3,2021,Tennessee Titans,12,379,1689,17,76,4.5,140.8,16,27.56
4,2021,Baltimore Ravens,11,359,1658,12,35,4.6,150.7,13,34.87


In [149]:
passing_d = scrape_table('https://www.pro-football-reference.com/years/2021/opp.htm', 'div_passing')
rushing_d = scrape_table('https://www.pro-football-reference.com/years/2021/opp.htm', 'div_rushing')

X:\Conda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """


In [150]:
passing_d.head()

,Year,Tm,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,...,Y/G,Rate,Sk,Yds.1,QBHits,TFL,Sk%,NY/A,ANY/A,EXP
0,2021,Buffalo Bills,11,212,368,57.6,1964,8,2.2,16,...,178.5,62.8,21,116,59,52,5.4,5.0,3.6,35.95
1,2021,Carolina Panthers,12,235,355,66.2,2120,17,4.8,8,...,176.7,91.7,32,259,71,61,8.3,5.5,5.4,1.63
2,2021,Arizona Cardinals,11,239,371,64.4,2244,14,3.8,8,...,204.0,86.5,29,176,65,48,7.3,5.6,5.4,19.63
3,2021,Los Angeles Chargers,11,224,343,65.3,2254,16,4.7,7,...,204.9,92.7,21,147,56,37,5.8,6.2,6.2,-52.61
4,2021,San Francisco 49ers,11,234,348,67.2,2280,15,4.3,5,...,207.3,95.8,24,169,49,61,6.5,6.1,6.3,-55.58


In [151]:
rushing_d.head()

,Year,Tm,G,Att,Yds,TD,Y/A,Y/G,EXP
0,2021,Tampa Bay Buccaneers,11,226,891,7,3.9,81.0,3.12
1,2021,Baltimore Ravens,11,228,926,10,4.1,84.2,21.72
2,2021,New Orleans Saints,11,296,1011,9,3.4,91.9,28.83
3,2021,Washington Football Team,11,251,1019,8,4.1,92.6,12.04
4,2021,Cincinnati Bengals,11,248,1031,9,4.2,93.7,10.96


In [152]:
import numpy as np
score_dict = {
    'Year':[],
    'Week':[],
    'Home':[],
    'Away':[],
    'Win':[],
    'Home_Score':[],
    'Away_Score':[]
}

team_set = set()

for i in range(1,19):
    page = requests.get(f'https://www.pro-football-reference.com/years/2021/week_{i}.htm').content
    soup = bs4.BeautifulSoup(page, 'html.parser')

    score_list = soup.findAll('table', {'class':'teams'})

    for item in score_list:
        text = np.array(item.get_text().strip().replace('\n', ' ').strip().split())
        if '2021' in text:
            text = text[3:]
        indexes = np.array([x.isdigit() for x in text]).nonzero()[0]
        if len(indexes)==2:
            score_dict['Year'].append(2021)
            score_dict['Week'].append(i)
            score_dict['Home'].append(text[indexes[0]-1])
            score_dict['Away'].append(text[indexes[1]-1])
            score_dict['Win'].append(int(text[indexes[0]]>text[indexes[1]]))
            score_dict['Home_Score'].append(text[indexes[0]])
            score_dict['Away_Score'].append(text[indexes[1]])
            team_set.add(text[indexes[0]-1])
        else:
            team_list = list(team_set.intersection(set(text)))
            score_dict['Year'].append(2021)
            score_dict['Week'].append(i)
            score_dict['Home'].append(team_list[0])
            score_dict['Away'].append(team_list[1])
            score_dict['Win'].append(np.nan)
            score_dict['Home_Score'].append(np.nan)
            score_dict['Away_Score'].append(np.nan)

scores_2021 = pd.DataFrame.from_dict(score_dict)

scores_2021

,Year,Week,Home,Away,Win,Home_Score,Away_Score
0,2021,1,Cowboys,Buccaneers,0.0,29,31
1,2021,1,Steelers,Bills,1.0,23,16
2,2021,1,Eagles,Falcons,0.0,32,6
3,2021,1,Chargers,Team,1.0,20,16
4,2021,1,Cardinals,Titans,1.0,38,13
...,...,...,...,...,...,...,...
267,2021,18,Lions,Packers,NaN,NaN,NaN
268,2021,18,Rams,49ers,NaN,NaN,NaN
269,2021,18,Raiders,Chargers,NaN,NaN,NaN
270,2021,18,Chiefs,Broncos,NaN,NaN,NaN


In [153]:
scores_2021.to_csv('nfl_scores_21.csv')
passing.to_csv('passing_21.csv')
passing_d.to_csv('passing_d_21.csv')
rushing.to_csv('rushing_21.csv')
rushing_d.to_csv('rushing_d_21.csv')